In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import random
from random import shuffle
from tqdm import tqdm

In [2]:
DATADIR = "D:\ML\PetImages"

CATEGORIES = ["Dog", "Cat"]

IMG_SIZE = 50
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            

create_training_data()

print(len(training_data))

100%|████████████████████████████████████████████████████████████████████████████| 12501/12501 [02:31<00:00, 82.52it/s]

24946


In [3]:
import random 
random.shuffle(training_data)

In [9]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)


In [10]:
X.shape

(24946, 50, 50, 1)

In [14]:
X[1]

array([[[105],
        [105],
        [100],
        ...,
        [121],
        [124],
        [101]],

       [[129],
        [122],
        [117],
        ...,
        [129],
        [125],
        [132]],

       [[117],
        [118],
        [117],
        ...,
        [ 63],
        [ 82],
        [ 89]],

       ...,

       [[ 57],
        [ 58],
        [ 61],
        ...,
        [116],
        [109],
        [105]],

       [[ 53],
        [ 50],
        [ 49],
        ...,
        [ 75],
        [106],
        [103]],

       [[ 45],
        [ 50],
        [ 44],
        ...,
        [107],
        [ 95],
        [ 94]]], dtype=uint8)

In [13]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
# !pip install tensorflow-gpu==1.15 

In [1]:
###### import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import pickle


X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))


X = X/255.0 

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=10, validation_split=0.3)

Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 14s 822us/sample - loss: 0.6327 - accuracy: 0.6329 - val_loss: 0.5770 - val_accuracy: 0.6978
Epoch 2/10
17462/17462 [==============================] - 7s 400us/sample - loss: 0.5316 - accuracy: 0.7353 - val_loss: 0.5823 - val_accuracy: 0.7028
Epoch 3/10
17462/17462 [==============================] - 7s 389us/sample - loss: 0.4774 - accuracy: 0.7726 - val_loss: 0.4726 - val_accuracy: 0.7758
Epoch 4/10
17462/17462 [==============================] - 7s 392us/sample - loss: 0.4370 - accuracy: 0.7965 - val_loss: 0.4842 - val_accuracy: 0.7782
Epoch 5/10
17462/17462 [==============================] - 7s 393us/sample - loss: 0.4034 - accuracy: 0.8154 - val_loss: 0.4461 - val_accuracy: 0.7878
Epoch 6/10
17462/17462 [==============================] - 7s 402us/sample - loss: 0.3701 - accuracy: 0.8327 - val_loss: 0.4391 - val_accuracy: 0.7982
Epoch 7/10
17462/17462 [==========================

In [17]:
model.save('dogs vs cats (model_2)')

INFO:tensorflow:Assets written to: dogs vs cats (model_2)\assets


In [4]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('tf_lite.pb')

In [5]:
CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value
import cv2

def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.
inputs = prepare(r'C:\Users\Gaming\Jupyter Notebook\Test Images\1.jpg')
prediction = loaded_model.predict(inputs)  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT

# prediction

if prediction == 0:
    predict = 'dog'
else:
    predict = 'cat'

print(predict)

cat


In [22]:
loaded_model
export_dir = r'C:\Users\Gaming\Jupyter Notebook\tf_lite.pb'

In [23]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [24]:
import pathlib
tflite_model_file =pathlib.Path(r'C:\Users\Gaming\Jupyter Notebook\model2.tflite')
tflite_model_file.write_bytes(tflite_model)

378320